In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os
import subprocess
from tqdm import tqdm

def get_last_date_from_csv(csv_file):
    """Get the last date from existing CSV file using system tail command"""
    if not os.path.exists(csv_file):
        return None
    
    try:
        # Use tail to get the last line
        result = subprocess.run(['tail', '-n', '1', csv_file], 
                              capture_output=True, text=True, check=True)
        last_line = result.stdout.strip()
        
        if not last_line:
            return None
        
        # Extract the first column (create_ts) from the last line
        first_column = last_line.split(',')[0]
        
        # Parse the timestamp and extract date
        last_datetime = pd.to_datetime(first_column)
        last_date = last_datetime.date()
        next_date = last_date + timedelta(days=1)
        
        print(f"Found existing data. Last date: {last_date}, resuming from: {next_date}")
        return next_date
        
    except Exception as e:
        print(f"Error reading last date with tail: {e}")
        return None

def download_kalshi_data(csv_file='kalshi_trade_data_2025.csv'):
    """Download Kalshi trade data with auto-resume using tail command"""
    
    # Determine start date
    start_date = get_last_date_from_csv(csv_file)
    if start_date is None:
        start_date = datetime(2025, 1, 1).date()
        print("Starting fresh download from 2025-01-01")
        file_mode = 'w'
        write_header = True
    else:
        file_mode = 'a'
        write_header = False
    
    # End date is yesterday
    end_date = (datetime.now() - timedelta(days=1)).date()
    
    if start_date > end_date:
        print("Already up to date!")
        return
    
    print(f"Downloading from {start_date} to {end_date}")
    
    base_url = 'https://kalshi-public-docs.s3.amazonaws.com/reporting/trade_data_{}.json'
    
    # Generate date range
    current_date = start_date
    dates_to_process = []
    while current_date <= end_date:
        dates_to_process.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)
    
    successful_count = 0
    failed_dates = []
    batch_data = []
    batch_size = 5  # Save every 5 successful downloads
    
    for date_str in tqdm(dates_to_process, desc="Downloading trade data"):
        url = base_url.format(date_str)
        
        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200:
                data = response.json()
                if data:  # Only process if data exists
                    df = pd.DataFrame(data)
                    
                    # Convert create_ts to timestamp
                    if 'create_ts' in df.columns:
                        df['create_ts'] = pd.to_datetime(df['create_ts'])
                    
                    # Reorder columns: create_ts, ticker_name, contracts_traded, price
                    desired_columns = ['create_ts', 'ticker_name', 'contracts_traded', 'price']
                    available_columns = [col for col in desired_columns if col in df.columns]
                    
                    if available_columns:
                        df = df[available_columns]
                        batch_data.append(df)
                        successful_count += 1
                        
                        # Save in batches
                        if len(batch_data) >= batch_size:
                            save_batch_to_csv(batch_data, csv_file, write_header and successful_count == len(batch_data))
                            batch_data = []
                            write_header = False
                    else:
                        failed_dates.append(date_str)
                        
        except Exception as e:
            failed_dates.append(date_str)
        
        time.sleep(0.05)  # Be nice to the server
    
    # Save any remaining batch data
    if batch_data:
        save_batch_to_csv(batch_data, csv_file, write_header and successful_count == len(batch_data))
    
    print(f"\nDownload complete!")
    print(f"Successfully processed: {successful_count} days")
    if failed_dates:
        print(f"Failed downloads: {len(failed_dates)} days")

def save_batch_to_csv(batch_data, csv_file, write_header):
    """Save a batch of dataframes to CSV"""
    if not batch_data:
        return
        
    combined_batch = pd.concat(batch_data, ignore_index=True)
    
    combined_batch.to_csv(
        csv_file, 
        mode='a' if not write_header else 'w',
        header=write_header,
        index=False
    )

def get_file_stats(csv_file='data/kalshi_trade_data_2025.csv'):
    """Get file statistics using system commands"""
    if not os.path.exists(csv_file):
        print(f"File {csv_file} not found")
        return
    
    try:
        # Get line count (subtract 1 for header)
        result = subprocess.run(['wc', '-l', csv_file], 
                              capture_output=True, text=True, check=True)
        line_count = int(result.stdout.split()[0]) - 1
        
        # Get first few lines to show sample
        result = subprocess.run(['head', '-n', '6', csv_file], 
                              capture_output=True, text=True, check=True)
        
        print(f"\nFile: {csv_file}")
        print(f"Total records: {line_count:,}")
        print(f"\nSample data:")
        print(result.stdout)
        
    except Exception as e:
        print(f"Error getting file stats: {e}")

In [6]:
download_kalshi_data()

Starting fresh download from 2025-01-01


  df['create_ts'] = pd.to_datetime(df['create_ts'])



Download complete!
Successfully processed: 217 days
